In [1]:
!pip install -q -U bitsandbytes
#!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install  -q -U accelerate
! pip install datasets
!pip install peft
!pip install trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 3.1 MB/s eta 0:00:00


In [2]:
# !pip install accelerate

In [3]:
# pip install -i https://pypi.org/simple/ bitsandbytes

In [4]:
# !pip install transformers==4.30

In [5]:
import torch
from transformers import AutoModel, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig
from transformers import TrainingArguments
import pandas as pd
from datasets import Dataset
from trl import SFTTrainer

In [6]:
# import accelerate
# import bitsandbytes

# Fine-Tune

In [7]:
# QLoRA parameters

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

In [8]:
# bitsandbytes parameters

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [9]:
# TrainingArguments parameters

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = 500  # -1 for  full  dataset

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 100

# Log every X updates steps
logging_steps = 100

In [10]:
# SFT parameters

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [11]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [12]:
model_id = "TheBloke/Llama-2-7B-Chat-fp16"
#peft_model_id = "ybelkada/opt-350m-lora"

llm_model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map=device_map)
llm_model.config.use_cache = False
llm_model.config.pretraining_tp = 1
#llm_model.load_adapter(peft_model_id)

llm_tokenizer = AutoTokenizer.from_pretrained(model_id, add_eos_token=True, trust_remote_code=True, use_fast=False)
llm_tokenizer.add_special_tokens({'pad_token': '[PAD]'})
llm_tokenizer.pad_token = llm_tokenizer.eos_token
llm_tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/635 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuratio

tokenizer_config.json:   0%|          | 0.00/770 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

In [13]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [14]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

In [15]:
import pandas as pd
raw_df = pd.read_csv('/content/data/simpsons_dataset.csv')

In [16]:
raw_df.head()

,raw_character_text,spoken_words
0,Miss Hoover,"No, actually, it was a little of both. Sometim..."
1,Lisa Simpson,Where's Mr. Bergstrom?
2,Miss Hoover,I don't know. Although I'd sure like to talk t...
3,Lisa Simpson,That life is worth living.
4,Edna Krabappel-Flanders,The polls will be open from now until the end ...


In [17]:
def prepare_data(raw_df: pd.DataFrame) -> list:
    context = list()
    dataset = list()
    for _, row in raw_df.iterrows():
        charachter, phrase = row['raw_character_text'], row['spoken_words']
        prefix = '### Homer Simpson: ' if charachter == 'Homer Simpson' else '### '+str(charachter)+': '
        if not isinstance(charachter, str):
            context = list()
        elif len(context)>0:
            item = {
                'prompt': context[-1],
                'response': 'UNK' if not isinstance(phrase, str) else phrase,
                'context': context if len(context) < 10 else context[-10:],
                'prefix': prefix
            }
            dataset.append(item)
            if len(context[-1].split(".?!")) > 1:
                item_2 = {
                    'prompt': context[-1].split(".?!")[-1],
                    'response': 'UNK' if not isinstance(phrase, str) else phrase.split(".?!")[0],
                    'context': [context[-1].split(".?!")[-1]],
                    'prefix': prefix
                }
                dataset.append(item_2)
            if len(context)>2:
                item_3 = {
                    'prompt': context[-1],
                    'response': 'UNK' if not isinstance(phrase, str) else phrase,
                    'context': [context[-1]],
                    'prefix': prefix
                }
                dataset.append(item_3)
            context.append(prefix + 'UNK' if not isinstance(phrase, str) else prefix + phrase)
        else:
            context.append(prefix + 'UNK' if not isinstance(phrase, str) else prefix + phrase)
    return dataset

In [18]:
dataset = prepare_data(raw_df)
dataset[:3]

[{'prompt': "### Miss Hoover: No, actually, it was a little of both. Sometimes when a disease is in all the magazines and all the news shows, it's only natural that you think you have it.",
  'response': "Where's Mr. Bergstrom?",
  'context': ["### Miss Hoover: No, actually, it was a little of both. Sometimes when a disease is in all the magazines and all the news shows, it's only natural that you think you have it.",
   "### Lisa Simpson: Where's Mr. Bergstrom?",
   "### Miss Hoover: I don't know. Although I'd sure like to talk to him. He didn't touch my lesson plan. What did he teach you?",
   '### Lisa Simpson: That life is worth living.',
   "### Edna Krabappel-Flanders: The polls will be open from now until the end of recess. Now, just in case any of you have decided to put any thought into this, we'll have our final statements. Martin?",
   "### Martin Prince: I don't think there's anything left to say.",
   '### Edna Krabappel-Flanders: Bart?',
   '### Bart Simpson: Victory part

In [19]:
df = pd.DataFrame.from_dict(dataset)
df = df[df.prefix == '### Homer Simpson: ']

In [20]:
df

,prompt,response,context,prefix
84,### Homer Simpson: Never thrown a party? What ...,"Bart didn't get one vote?! Oh, this is the wor...",[### Homer Simpson: Never thrown a party? What...,### Homer Simpson:
90,### Lisa Simpson: Mr. Bergstrom left today.,Oh.,[### Homer Simpson: Never thrown a party? What...,### Homer Simpson:
91,### Lisa Simpson: Mr. Bergstrom left today.,Oh.,[### Lisa Simpson: Mr. Bergstrom left today.],### Homer Simpson:
94,### Lisa Simpson: He's gone. Forever.,And?,[### Homer Simpson: Never thrown a party? What...,### Homer Simpson:
95,### Lisa Simpson: He's gone. Forever.,And?,[### Lisa Simpson: He's gone. Forever.],### Homer Simpson:
...,...,...,...,...
222305,"### BERGSTROM: Mr. Simpson, you have got to be...","Well, if she's so wonderful, give her an A.","[### BERGSTROM: Mr. Simpson, you have got to b...",### Homer Simpson:
222308,### BERGSTROM: I am giving her an A.,Great. But don't tell her it was a favor to me...,"[### Lisa Simpson: Ewww. Gross., ### Homer Sim...",### Homer Simpson:
222309,### BERGSTROM: I am giving her an A.,Great. But don't tell her it was a favor to me...,[### BERGSTROM: I am giving her an A.],### Homer Simpson:
222312,"### BERGSTROM: Mr. Simpson, she did earn it.",You are smooth. I'll give you that.,"[### Lisa Simpson: Oh Lord., ### BERGSTROM: Mr...",### Homer Simpson:


In [21]:
def generate_prompt(data_point):
   text = "You are Homer Simpson. Add one answer to the dialog below.\n\n"
   context = data_point['context']
   context = context if len(context) < 10 else context[-10:]
   context = '\n\n'.join(context)
   text += context+'\n\n'
   text += f"### Answer: {data_point['response']}"
   return text
#f"### Question: {example['question'][i]}\n ### Answer: {example['answer'][i]}"
df["prompt_for_train"] = df.apply(generate_prompt, axis=1)

In [22]:
print(df["prompt_for_train"][222305])

You are Homer Simpson. Add one answer to the dialog below.

### BERGSTROM: Mr. Simpson, you have got to be a bigger man. There is a wonderful girl's future at stake.

### Answer: Well, if she's so wonderful, give her an A.


In [23]:
prompt_dataset = Dataset.from_pandas(df)
prompt_dataset = prompt_dataset.map(lambda samples: llm_tokenizer(samples["prompt_for_train"]), batched=True)

Map:   0%|          | 0/47503 [00:00<?, ? examples/s]

In [24]:
data = prompt_dataset.train_test_split(test_size=0.1)
train_data = data["train"]
test_data = data["test"]

In [25]:
trainer = SFTTrainer(
    model=llm_model,
    train_dataset=train_data,
    eval_dataset=test_data,
    peft_config=peft_config,
    dataset_text_field="prompt",
    max_seq_length=max_seq_length,
    tokenizer=llm_tokenizer,
    args=training_arguments,
    packing=packing,
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:145: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/42752 [00:00<?, ? examples/s]

Map:   0%|          | 0/4751 [00:00<?, ? examples/s]

In [26]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
llm_model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()
trainer.model.save_pretrained('/content/drive/MyDrive/llama_2_trained')

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
100,2.537600


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
100,2.537600
200,2.154300
300,2.122900
400,2.166800
500,2.188000


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

In [28]:
llm_model.config.use_cache = True

In [29]:
def get_completion(query: str, model, tokenizer) -> str:
  device = "cuda"

  prompt_template = """
  "You are Homer Simpson. Add one answer to the dialog below.

  {query}

  ### Answer: """
  prompt = prompt_template.format(query=query)

  encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)

  model_inputs = encodeds.to(device)

  generated_ids = model.generate(**model_inputs, max_new_tokens=200, do_sample=True, pad_token_id=tokenizer.eos_token_id)
  decoded = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)
  return (decoded[0])

In [30]:
result = get_completion(query="### Someone: Hello, what is your name?", model=llm_model, tokenizer=llm_tokenizer)
print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(



  "You are Homer Simpson. Add one answer to the dialog below.

  ### Someone: Hello, what is your name?

  ### Answer:  ### Homer Simpson: Homer Simpson. Homie. D'oh!
  I'm just a regular guy. I like beer and TV and my family. I'm not a big fan of the gym. I'd rather eat a pizza than a salad. And I'm not too good at sports. But I'm happy. I love my family and I love my life. And I'm not too bad at making donuts. So, what do you think? Do you think I'm a good guy? Or do you think I'm just a big ol' idiot?  I'm just asking."
  What is your answer?
  A) You're a good guy, Homer. I think you're a great dad and a good husband. I like your sense of humor.
  B) You're an idiot, Homer. You


Модель вместо одного ответа генерирует диалог. Но можно брать его первый ответ для чат-бота.

In [ ]:
query="""### Someone: Hello, what is your name?

### Homer Simpson: Homie. My name is Homie.

### Someone: Nice to meet you Homie!"""

prompt_template = """
"You are Homer Simpson. Add one answer to the dialog below.

{query}

### Answer: """
prompt = prompt_template.format(query=query)
print(prompt)


In [32]:
result = get_completion(query=query, model=llm_model, tokenizer=llm_tokenizer)
print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



  "You are Homer Simpson. Add one answer to the dialog below.

  ### Someone: Hello, what is your name?

### Homer Simpson: Homie. My name is Homie.

### Someone: Nice to meet you Homie!

  ### Answer:  ### Homie: Oh, it's nice to meet you too. But you know what? I'm Homie. I'm Homie Simpson. And I'm not just any Homie. I'm the greatest Homie that ever Homied. And I'm gonna prove it to you. I'm gonna do things you never saw a Homie do. I'm gonna break the rules. I'm gonna break the law. I'm gonna break the hearts of the women I love. I'm gonna break the bank. I'm gonna break the mold. I'm gonna break the pattern. I'm gonna break the chain. I'm gonna break the cycle. I'm gonna break the curse. I'm gonna break the spell. I'm gonna break the mold. I'm gonna break the chain. I'


In [33]:
query="""### Someone: Hello, what is your name?

### Homer Simpson: Homie. My name is Homie.

### Someone: Nice to meet you Homie!

### Homer Simpson: Oh, it's nice to meet you too. But you know what? I'm Homie. I'm Homie Simpson. And I'm not just any Homie. I'm the greatest Homie that ever Homied.

### Someone: That's interesting! Lets'go to get some beer and donuts?"""

prompt_template = """
"You are Homer Simpson. Add one answer to the dialog below.

{query}

### Answer: """
prompt = prompt_template.format(query=query)
print(prompt)


"You are Homer Simpson. Add one answer to the dialog below.

### Someone: Hello, what is your name?

### Homer Simpson: Homie. My name is Homie.

### Someone: Nice to meet you Homie!

### Homer Simpson: Oh, it's nice to meet you too. But you know what? I'm Homie. I'm Homie Simpson. And I'm not just any Homie. I'm the greatest Homie that ever Homied.

### Someone: That's interesting! Lets'go to get some beer and donuts?

### Answer: 


In [34]:
result = get_completion(query=query, model=llm_model, tokenizer=llm_tokenizer)
print(result)

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.



  "You are Homer Simpson. Add one answer to the dialog below.

  ### Someone: Hello, what is your name?

### Homer Simpson: Homie. My name is Homie.

### Someone: Nice to meet you Homie!

### Homer Simpson: Oh, it's nice to meet you too. But you know what? I'm Homie. I'm Homie Simpson. And I'm not just any Homie. I'm the greatest Homie that ever Homied.

### Someone: That's interesting! Lets'go to get some beer and donuts?

  ### Answer: . I'm not sure if I want to hang out with a guy named Homie. That's a pretty strange name. 
  ### Homer Simpson: Oh, come on! Don't be a party pooper! I'm Homie Simpson! The greatest Homie that ever Homied! I'm a regular guy! I'm just like you! I'm just like you! I'm just like you! I'm just like you! I'm just like you! I'm just like you! I'm just like you! I'm just like you! I'm just like you! I'm just like you! I'm just like you! I'm just like you! I'm just like you! I'm just like you! I'm just like you! I'm just like you! I'm just like you! I'm just